In [6]:
import pandas as pd
import faiss
import numpy as np
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import os
from sentence_transformers import SentenceTransformer

In [ ]:
df = pd.read_csv('C:/course-descriptions.csv')
df.head()

,Course ID,Title,Description
0,1001,Architecting Big Data applications,Learn how to architect both simple and complex...
1,1002,MLOps Essentials: Monitoring Model Drift and Bias,As more and more ML models are developed and d...
2,1003,Apache Kafka Essential Training: Getting Started,"In this course, instructor introduces Apache..."
3,1004,Applied AI: Building NLP Apps with Hugging Fac...,Explore models designed for common NLP use cas...
4,1005,Deep Learning : Getting started,Deep learning as a technology has grown leaps ...


In [7]:
# Initialize sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\1090135\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\1090135\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
course_description = df['Description'].tolist()

In [10]:
embedding = model.encode(course_description, convert_to_numpy=True)
print(f'embedding size : {embedding.shape[1]}')

embedding size : 384


In [11]:
print(f'embedding size : {embedding.shape}')

embedding size : (5, 384)


# Store Embeddings with Faiss

In [14]:
# 384 is the size of the vector
dimension = embedding.shape[1]

index = faiss.IndexFlatL2(dimension)

ADD the embedding to the faiss index

In [15]:
index.add(embedding)

# Searching with Faiss

In [17]:
Query = 'What is the best course for deeplearning?'

Query_embedding = model.encode([Query], convert_to_numpy=True)

k = 3

distance, indices = index.search(Query_embedding, k)

In [18]:
print(f'index : {indices}')

index : [[4 0 3]]


In [19]:
df.columns

Index(['Course ID', 'Title', 'Description'], dtype='object')

In [23]:
print('top 3 matching course : ')
for i in range(k):
    print(f"courseid : {df['Course ID'].iloc[indices[0, i]]}, distance is : {distance[0][i]}")

top 3 matching course : 
courseid : 1005, distance is : 0.958468496799469
courseid : 1001, distance is : 1.6306434869766235
courseid : 1004, distance is : 1.6766505241394043


In [26]:
print('top 3 matching course : ')
for i in range(k):
    print(f"courseid : {df['Title'].iloc[indices[0][i]]}")

top 3 matching course : 
courseid : Deep Learning : Getting started
courseid : Architecting Big Data applications
courseid : Applied AI: Building NLP Apps with Hugging Face Transformers
